In [108]:
#Importing...

import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize



#Reading the .mat file

mat_contents = sio.loadmat('E:\Machine Learning\Exercises\machine-learning-ex3\ex3\ex3data1.mat')

mat_contents

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [109]:
#Taking matrix X and Y
X = mat_contents['X']

m,n = X.shape
x0 = [1 for i in range(m)]

X = np.concatenate((np.array(x0).reshape(m,1),X),axis = 1)
print(X.shape)


Y = mat_contents['y']
print(Y)

(5000, 401)
[[10]
 [10]
 [10]
 ...
 [ 9]
 [ 9]
 [ 9]]


In [110]:
#INITIALIZING...

input_layer_size  = 400  # 20x20 Input Images of Digits

num_labels = 10          # 10 labels, from 1 to 10
                         # (note that we have mapped "0" to label 10)
print(X.shape)

Lambda = 0.1

(5000, 401)


In [111]:
def sigmoid(z):
    
    #IF z IA AN ARRAY
    if isinstance(z,np.ndarray):
        temp = np.ones(z.shape)
       
        temp = 1/(1 + np.exp(-z))
        
        return temp
   
    #IF z IS A LIST
    elif isinstance(z,list):
        for i in range(len(z)):
            z[i] = 1/(1 + np.exp(-z[i]))
        return z
    
    #IF z IS A NO.   
    else:
        return 1/(1 + np.exp(-z))


In [112]:
#PREVIOUS FUNCTION FOR USING fmincg

#calculating the cost function
def costFunction(theta,Xmapped,Y,Lambda):
    
    #no. of dataset and features for the testing data
    m,n = Y.shape
    
    #first term of a cost function ylog(h(x))
    firstTerm = ((Y.transpose()).dot(np.log(sigmoid(Xmapped.dot(theta)))))
    
    #second term of a cost function (1-y)log(1-h(x))
    secondTerm = ((1-Y).transpose()).dot(np.log(1-sigmoid((Xmapped.dot(theta)))))
   
    #creating third term : lambda.theta^2/2m
    """We had minus the theta[0] here because we do not have to include theta[0]
                                                because loop starts from the 1 & theta[0] is not inluded in regularized
                                                parameter"""
    thirdTerm = (Lambda/(2*m))*(((theta**2).sum()) - theta[0]**2)
    return  (thirdTerm - (((firstTerm+secondTerm))/m))
    
    
#FOR GRADIENT
    
def Gradient(theta,Xmapped,Y,Lambda):
    
    #no. of dataset and features for the testing data
    m,n = Y.shape
    
    #storing the value of theta[0] so that it will not be change by (lambda/m)*theta[0]
    theta0 = theta[0]
    
    #performing the gradient over all theta
    theta = (((Xmapped.transpose()).dot(sigmoid(Xmapped.dot(theta))-Y)/m)) +(Lambda/m)*theta
    
    #since in theta[0] extra term (lambda/m)*theta[0] is added so we have to subtract from it
    theta[0] = theta[0] - (Lambda/m)*theta0

    
    # returning the total cost and gradient
    return theta

print(n)

400


In [113]:
"""def oneVsAll(X,Y,num_labels,Lambda):
    m,n = X.shape
    
    all_theta = np.zeros((num_labels,n))
    
    for i in range(1,num_labels+1):
        Ytemp = Y.copy()
        Y1 = (Y != i)
        Ytemp[Y1] = 0
        
        Y1 = (Y == i)
        Ytemp[Y1] =1
        
        theta = np.zeros((1,n))
        res = (minimize(costFunction, theta,args = (X,Ytemp,Lambda), method='BFGS',options = {'maxiter':500,'disp':True }))
        all_theta = np.concatenate((all_theta,(res.x).reshape(1,n)),axis = 0)
        print(all_theta[i-1])
    return all_theta

all_theta = oneVsAll(X,Y,num_labels,Lambda).shape"""

#computing one vs all

m,n = X.shape

#giving a row of zero to all_theta
all_theta = np.zeros((1,n))
    
for i in range(1,num_labels+1):
    
    #using Indexing with boolean arrays
    
    #isme hmne hr ek classifier k liye Ytemp bnaya i.e, for ith no. Y matrix me jha jha i milega use vo 1 se or baki ko 0 se
    #replace kr dega this is done to make it as a separatre binary classifier
    Ytemp = Y.copy()
    Y1 = (Y != i)
    Ytemp[Y1] = 0
        
    Y1 = (Y == i)
    Ytemp[Y1] =1
      
    #theta_initial
    theta = np.zeros((1,n))
    
    res = (minimize(costFunction, theta,args = (X,Ytemp,Lambda), method='BFGS',options = {'maxiter':100,'disp':True }))
    all_theta = np.concatenate((all_theta,(res.x).reshape(1,n)),axis = 0)
    print(all_theta[i])



         Current function value: 0.015886
         Iterations: 100
         Function evaluations: 40703
         Gradient evaluations: 101
[-3.72644880e+00 -4.22245806e-09 -4.22245806e-09  5.15661920e-05
 -4.77421857e-04 -5.29120556e-04  5.15355529e-04 -8.44307576e-03
  2.93788089e-03  2.34854152e-04  6.15957611e-04  1.28893072e-04
  1.03153823e-04  3.48323578e-04  1.03534484e-03  5.72065873e-03
  7.78456098e-03  6.54028753e-04 -1.10481594e-04 -1.46392091e-07
 -4.22245806e-09  1.14835664e-08  9.19475171e-06 -6.84258292e-04
  5.28836906e-03  6.20230673e-03  1.40246459e-03 -1.10396043e-02
  2.89917012e-03 -3.54732623e-03 -1.12294653e-02 -1.72923928e-03
 -7.56579962e-04 -2.84136472e-03 -4.93758929e-03 -5.46913428e-02
 -7.73603480e-02  2.76257541e-04  5.71570560e-03  3.45544134e-04
  1.55201288e-04  2.01591079e-07 -1.90361719e-04  5.06328815e-04
  1.57649126e-02  3.07452063e-02  3.81157354e-02  3.82793978e-02
 -5.69496705e-02 -7.86894008e-02 -3.14024135e-02 -3.00195097e-02
 -2.12491211e-03

         Current function value: 0.063871
         Iterations: 100
         Function evaluations: 40703
         Gradient evaluations: 101
[-5.78627856e+00  0.00000000e+00  0.00000000e+00 -8.94921270e-06
 -1.45215076e-04  3.77036303e-03  1.75812137e-02 -2.01493446e-02
 -7.20939082e-03 -1.87270390e-03 -1.43222040e-02 -4.45842776e-03
 -3.68600282e-03 -1.48582734e-02  2.19527617e-02  3.56034174e-02
 -2.54812143e-03 -9.77159033e-04 -2.06092424e-05  1.00267694e-05
  0.00000000e+00 -8.34785813e-08  6.55777497e-06  5.00894572e-04
 -4.46411801e-03 -1.52524393e-02 -4.68891440e-02 -1.18202872e-01
 -8.92337946e-02 -8.28365388e-02 -8.76574051e-02 -5.05848154e-02
  5.22681257e-02  1.42664721e-01 -2.93011785e-01 -4.24605340e-01
  1.74086411e-02 -1.42503033e-02 -3.94445424e-02 -5.74048338e-03
  7.14363099e-04  3.88769105e-07 -1.85313685e-04 -8.33356663e-03
  7.88501047e-02  4.76507418e-01  1.00268405e+00  8.51086451e-01
  3.88633994e-01 -1.22760162e-01 -1.00612423e-01 -1.84738788e-01
 -2.60322831e-01

         Current function value: 0.062642
         Iterations: 100
         Function evaluations: 40703
         Gradient evaluations: 101
[ 8.90146309e-01  0.00000000e+00  0.00000000e+00 -1.21663206e-06
  6.59533538e-05 -6.23714300e-04 -5.95873287e-03 -3.64338898e-03
 -1.52499624e-02 -8.04175624e-03 -1.31424471e-03  5.41034508e-05
  1.20653779e-03  6.03819381e-03 -3.20207367e-02 -5.00227479e-02
 -6.30783903e-05  7.69281579e-04  4.84953166e-06 -4.75111773e-06
  0.00000000e+00 -4.30269092e-07 -8.29740704e-06 -5.55759747e-04
  5.76898940e-03  1.91016097e-02  2.65515158e-02 -4.49691363e-03
 -7.51521167e-02 -4.41011755e-02 -2.16948552e-02  2.34749655e-03
 -1.00076067e-02 -6.87768647e-02  3.47486380e-01  5.51859638e-01
 -1.80693552e-03 -9.59499469e-03 -3.01394525e-03  2.23288576e-04
  2.84586652e-05  3.38300783e-06 -4.49179010e-05  8.18043208e-03
 -5.84705001e-02 -2.96895967e-01 -6.71084531e-01 -8.14538445e-01
 -2.53864584e-01 -1.15170774e-01  9.43826517e-03  1.45238682e-02
 -3.40025223e-02

         Current function value: 0.036121
         Iterations: 100
         Function evaluations: 40703
         Gradient evaluations: 101
[-2.94347785e+00  0.00000000e+00  0.00000000e+00 -4.18662731e-05
  3.27349615e-04  8.42733730e-04  5.18230743e-03  3.56685841e-02
  6.98082155e-05 -1.30260200e-03 -7.69924800e-03 -2.89294593e-03
  7.65437711e-05  1.45543165e-04 -6.72324474e-03 -2.39488909e-03
  4.35643890e-04  4.24932170e-05  3.59922323e-06 -7.41970026e-07
  0.00000000e+00 -1.97936892e-08 -4.06470061e-07  4.34308185e-04
 -3.37069649e-03 -1.00355514e-02 -8.14685157e-02  1.08219017e-01
  2.86797875e-01  1.80634666e-01  1.08500980e-01  6.11133525e-03
 -3.44390703e-02 -4.26855773e-02  7.08963583e-02  4.25398023e-02
  8.96622532e-03  1.50469983e-02  9.54033127e-03 -1.61379165e-03
  3.35644151e-05  3.71492823e-07 -3.60859728e-06  6.16584442e-05
 -3.59826255e-04 -1.76136439e-02  1.96107105e-01  2.90943230e-01
  2.08387328e+00  2.98040713e+00  1.90328264e+00  9.17089001e-01
  9.20865661e-01

         Current function value: 0.077986
         Iterations: 100
         Function evaluations: 40703
         Gradient evaluations: 101
[-5.55667877e+00  0.00000000e+00  0.00000000e+00 -2.48625065e-06
  2.00416876e-05  6.38430807e-04 -7.57074357e-03 -6.13508836e-03
 -1.20159475e-03 -2.10765473e-02 -1.27128427e-02 -6.41468143e-04
  1.27664675e-03  1.76035626e-03  3.85509459e-03  4.06119864e-03
  1.27637145e-03  1.16341665e-03  1.59559670e-04 -2.31761673e-05
  0.00000000e+00 -1.12124728e-07 -7.88160486e-06 -9.48897573e-06
  1.59947002e-03 -5.89353317e-03 -3.96490547e-03 -2.06124271e-02
 -1.18183545e-01 -2.31081259e-01 -8.94686104e-02 -1.02544770e-02
 -2.37498921e-02 -2.13841267e-02 -2.30820596e-02 -2.92361161e-02
 -8.88324182e-03 -8.51993822e-03  3.87218846e-03 -1.39148109e-03
 -2.21347500e-03  1.17801023e-06  2.97971497e-05  3.41218071e-04
 -3.88586599e-03 -6.27348182e-02 -1.84478143e-01 -2.53232988e-01
 -1.18171807e+00 -1.84481065e+00 -6.30515521e-01 -1.24015996e-01
 -2.15368799e-01

In [114]:
print(X.shape)

(5000, 401)


In [115]:
print(X[1])

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.22856793e-04 -9.15576210e-04
 -2.53532265e-02 -3.85931128e-02 -3.84044678e-02 -1.93746688e-02
  3.90933920e-04  8.19371408e-05  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [116]:
a = np.arange(20).reshape(5,4)
b = (a!=3)
a[b]=1

b = (a == 3)
a[b] = 0

print(a)

[[1 1 1 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]


In [117]:
print(X.shape)

(5000, 401)


In [118]:
"""from scipy.optimize import fmin_cg



m,n = X.shape
    
all_theta = np.zeros((1,n))
    
for i in range(1,num_labels+1):
    Ytemp = Y.copy()
    Y1 = (Y != i)
    Ytemp[Y1] = 0
        
    Y1 = (Y == i)
    Ytemp[Y1] =1
        
    theta = np.zeros((1,n))
    res = fmin_cg(costFunction, theta,args = (X,Ytemp,Lambda),maxiter = 400,full_output = 1,disp=1 )
    all_theta = np.concatenate((all_theta,(res.xopt).reshape(1,n)),axis = 0)
    print(all_theta[i-1])
    return all_theta


"""

'from scipy.optimize import fmin_cg\n\n\n\nm,n = X.shape\n    \nall_theta = np.zeros((1,n))\n    \nfor i in range(1,num_labels+1):\n    Ytemp = Y.copy()\n    Y1 = (Y != i)\n    Ytemp[Y1] = 0\n        \n    Y1 = (Y == i)\n    Ytemp[Y1] =1\n        \n    theta = np.zeros((1,n))\n    res = fmin_cg(costFunction, theta,args = (X,Ytemp,Lambda),maxiter = 400,full_output = 1,disp=1 )\n    all_theta = np.concatenate((all_theta,(res.xopt).reshape(1,n)),axis = 0)\n    print(all_theta[i-1])\n    return all_theta\n\n\n'

In [119]:
#print((sigmoid(X.dot(all_theta.transpose()))).max(), '   ',Y)
all_theta = np.delete(all_theta,0,0)
print(all_theta)

[[-3.72644880e+00 -4.22245806e-09 -4.22245806e-09 ...  2.60582675e-03
   1.08018767e-07  0.00000000e+00]
 [-3.91620178e+00  0.00000000e+00  0.00000000e+00 ...  4.35769349e-03
  -4.99832552e-04  0.00000000e+00]
 [-5.78627856e+00  0.00000000e+00  0.00000000e+00 ... -3.53452687e-05
  -3.05959142e-07  0.00000000e+00]
 ...
 [-9.07931658e+00  0.00000000e+00  0.00000000e+00 ... -1.12609984e-04
   9.64002370e-06  0.00000000e+00]
 [-5.55667877e+00  0.00000000e+00  0.00000000e+00 ... -7.12711189e-04
   5.09572109e-05  0.00000000e+00]
 [-7.68708716e+00  0.00000000e+00  0.00000000e+00 ... -4.65941884e-04
   1.50827777e-05  0.00000000e+00]]


In [120]:
#PREDICTING the hypothesis it will be a (5000,11) matrix jisme ki probabilities hongi

predict = sigmoid(X.dot(all_theta.transpose()))#is matrix me ek row ka hr column element probability bta raha hai us no.(1-10)
                                                #k hone ki jo ki corresponding index se pta lg raha hai
count = 0

r,c = predict.shape
print(predict.shape)
for i in range(r):
    
    #ab hmare pas probabilties hai predict variable me hr row ka max element jha hai uska (index+1) no. signify krta hai us
    #row pe jonsa hoga
    if (np.argmax(predict[i])+1)==int(Y[i]):
        count+=1
print(count)

(5000, 10)
4747


In [121]:
#Predicting the percentage accuracy

print("The percentage prediction is : ",count*100/r,' % ')

The percentage prediction is :  94.94  % 
